Billy CVAE ATTEMPT 2022/10/14 - DEVELOPED FROM Zooey's VAE

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import tensorflow as tf
import tensorflow_probability as tfp
import h5py
import random
from sklearn.model_selection import train_test_split
import pandas as pd
import photoz_utils

In [2]:
tfpl = tfp.layers
tf1 = tf.compat.v1
tfd = tfp.distributions

In [3]:
from tensorflow_probability.python.distributions import kl_divergence

In [4]:
gpus = tf.config.list_physical_devices('GPU')

if gpus:
  # Restrict TensorFlow to only allocate 10GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit = 5000)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [5]:
n = 10000
t = 2000
v = 2000

In [6]:
LATENT_DIM = 16
BASE_DEPTH = 32
IMAGE_SHAPE = (5, 127, 127)
BATCH_SIZE = 128
EPOCHS = 100
GB_LIMIT = 10
CHECKPOINTS_TO_SAVE = 4
KL_WEIGHT = 0.1

In [7]:
hf_train = h5py.File('/data/HSC/HSC_v6/step2A/127x127/five_band_image127x127_with_metadata_corrected_training_small.hdf5', 'r')
hf_test = h5py.File('/data/HSC/HSC_v6/step2A/127x127/five_band_image127x127_with_metadata_corrected_testing_small.hdf5', 'r')
hf_validation = h5py.File('/data/HSC/HSC_v6/step2A/127x127/five_band_image127x127_with_metadata_corrected_validation_small.hdf5', 'r')
x_train = np.asarray(hf_train['image']).reshape(n, np.prod(IMAGE_SHAPE))
x_test = np.asarray(hf_test['image']).reshape(t, np.prod(IMAGE_SHAPE))
x_validation = np.asarray(hf_validation['image']).reshape(v, np.prod(IMAGE_SHAPE))
max_value = 4.16
x_train = np.true_divide(x_train, max_value)
x_test = np.true_divide(x_test, max_value)
x_validation = np.true_divide(x_validation, max_value)
y_train = np.asarray(hf_train['specz_redshift'][0:])[..., None]
y_test = np.asarray(hf_test['specz_redshift'][0:])[..., None]
y_validation = np.asarray(hf_validation['specz_redshift'][0:])[..., None]
object_id_train = np.asarray(hf_train['object_id'][0:])
object_id = np.asarray(hf_test['object_id'][0:])
object_id_validation = np.asarray(hf_validation['object_id'][0:])
hf_train.close()
hf_test.close()
hf_validation.close()

In [8]:
def to_image(array):
    split_1 = np.array_split(array, 5)
    print(split_1[0].shape)
    im = np.array((np.array_split(split_1[0], 127), np.array_split(split_1[1], 127), np.array_split(split_1[2], 127), np.array_split(split_1[3], 127), np.array_split(split_1[4], 127))) 
    print(im.shape)
    return im

In [9]:
IMAGE_DIM = np.prod(IMAGE_SHAPE)

In [10]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Cropping2D, ZeroPadding2D
from tensorflow.keras.layers import Conv2D, Conv2DTranspose
from tensorflow.keras.layers import Input, Flatten, Dense, Reshape
import tensorflow_probability as tfp
from tensorflow_probability import layers as tfpl
from tensorflow_probability import distributions as tfd
from tensorflow.nn import leaky_relu

In [11]:
DATASET_NAME = "HSC_v6_small"
MODEL_TYPE = "CVAE"
MODEL_VERSION = "v0"

model_id = '_'.join([DATASET_NAME, MODEL_TYPE, MODEL_VERSION])
dir_model = os.path.join('/models', model_id, 'model')
dir_checkpoints = os.path.join('/models', model_id, 'checkpoints')
dir_logs = os.path.join('/logs', model_id)
dir_predictions = os.path.join('/predictions', model_id)
weights_file = dir_model + '/weights.h5'

print(weights_file)
print(dir_model)
print(dir_checkpoints)
print(dir_logs)
print(dir_predictions)

os.makedirs(dir_model, exist_ok=True)
os.makedirs(dir_checkpoints, exist_ok=True)
os.makedirs(dir_logs, exist_ok=True)
os.makedirs(dir_predictions, exist_ok=True)

/models/HSC_v6_small_CVAE_v0/model/weights.h5
/models/HSC_v6_small_CVAE_v0/model
/models/HSC_v6_small_CVAE_v0/checkpoints
/logs/HSC_v6_small_CVAE_v0
/predictions/HSC_v6_small_CVAE_v0


In [12]:
prior = tfd.Independent(tfd.Normal(loc=tf.zeros(LATENT_DIM), scale=1), reinterpreted_batch_ndims=1)

In [15]:
class CVAE(Model):
    def __init__(self):
        super(CVAE, self).__init__()
        
        images = Input(shape=IMAGE_SHAPE)
        labels = Input(shape=(1,))
        c1 = Conv2D(BASE_DEPTH, 3, strides=1, activation=leaky_relu,
                   padding='same', data_format='channels_first')(images)
        c2 = Conv2D(BASE_DEPTH, 3, strides=2, activation=leaky_relu,
                   padding='same', data_format='channels_first')(c1)
        c3 = Conv2D(2*BASE_DEPTH, 3, strides=1, activation=leaky_relu,
                   padding='same', data_format='channels_first')(c2)
        c4 = Conv2D(2*BASE_DEPTH, 3, strides=2, activation=leaky_relu,
                   padding='same', data_format='channels_first')(c3)
        c5 = Conv2D(4*LATENT_DIM, 3, strides=1, activation=leaky_relu,
                   padding='same', data_format='channels_first')(c4)
        ft = Flatten()(c5)
        d1 = Dense(200, activation=None)(ft)
        d2 = Dense(200, activation = leaky_relu)(labels),
        d3 = Dense(200, activation=None)(d2)
        concat = tf.keras.layers.Concatenate()([d1, d3])
        d4 = Dense(tfpl.MultivariateNormalTriL.params_size(LATENT_DIM), activation=None)(concat)
        z = tfpl.MultivariateNormalTriL(LATENT_DIM,
                  activity_regularizer=tfpl.KLDivergenceRegularizer(prior, weight=KL_WEIGHT))(d4)
        self.encoder = Model(inputs=[images, labels], outputs = [z], name='encoder')

        latents = Input(shape=LATENT_DIM)
        x = Dense(4*LATENT_DIM*32*32, activation=None)(latents)
        x = Reshape((4*LATENT_DIM,32,32))(x)
        x = Conv2DTranspose(4*LATENT_DIM, 3, strides=1, activation=leaky_relu,
                            padding='same', data_format='channels_first')(x)
        x = Conv2DTranspose(2*BASE_DEPTH, 3, strides=1, activation=leaky_relu,
                            padding='same', data_format='channels_first')(x)
        x = Conv2DTranspose(2*BASE_DEPTH, 3, strides=2, activation=leaky_relu,
                            padding='same', data_format='channels_first')(x)
        x = Conv2DTranspose(BASE_DEPTH, 3, strides=1, activation=leaky_relu,
                            padding='same', data_format='channels_first')(x)
        x = Conv2DTranspose(BASE_DEPTH, 3, strides=2, activation=leaky_relu,
                            padding='same', data_format='channels_first')(x)
        x = Conv2DTranspose(BASE_DEPTH, 3, strides=1, activation=leaky_relu,
                            padding='same', data_format='channels_first')(x)
        x = Conv2D(IMAGE_SHAPE[0], 3, strides=1, activation=None, 
                   padding='same', data_format='channels_first')(x)
        outputs = Cropping2D(cropping=((0,1),(0,1)), data_format='channels_first')(x)
        self.decoder = Model(latents, outputs, name='decoder')

    def call(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x
    
    def summary(self):
        print(self.encoder.summary())
        print(self.decoder.summary())

In [16]:
model = CVAE()

TypeError: Dimension value must be integer or None or have an __index__ method, got value 'TensorShape([None, 200])' with type '<class 'tensorflow.python.framework.tensor_shape.TensorShape'>'